# Abalone Age Prediction by Multiclass Classification using ANN (testing)

## Load dataset for the training process

In [1]:
import numpy as np
import pandas as pd

### Open the CSV

In [2]:
df_test = pd.read_csv("abalone_test.csv")
print("Num test:", len(df_test))

Num test: 835


### Create dataset for testing process

In [3]:
import tensorflow as tf

2023-05-06 20:34:36.689545: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-06 20:34:36.691376: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-06 20:34:36.769083: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-06 20:34:36.797956: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-06 20:34:38.539986: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

### Features to be used and target

In [4]:
x_names = ["Sex", "Length", "Diameter", "Height", "Whole weight", "Shucked weight", "Viscera weight", "Shell weight"]
y_name = "Rings"
y_classes = list(range(1, 30)) # 1 to 29 (from EDA)

### Convert Pandas dataframe to TF dataset

In [5]:
def df_to_dataset(df: pd.DataFrame) -> tf.data.Dataset:
    features = df[x_names].to_numpy(dtype=np.float32)
    labels = df[[y_name]].to_numpy(dtype=np.int64).squeeze(axis=1)
    labels = labels - 1 # map class #0 to Rings 1
    my_dataset = tf.data.Dataset.from_tensor_slices((features, labels))
    return my_dataset

In [6]:
ds_test = df_to_dataset(df_test)

2023-05-06 20:34:41.001892: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-06 20:34:41.002623: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


### Use the dataset

In [7]:
BATCH_SIZE = 64
ds_test = ds_test.batch(BATCH_SIZE)

## Model testing

In [8]:
from tqdm.notebook import tqdm
from sklearn.metrics import f1_score, accuracy_score

### Which device we will use for training process (CPU/GPU)

If a TensorFlow operation has both CPU and GPU implementations, by default, the GPU device is prioritized when the operation is assigned. So, no need to manually specify here.

*Reference: [Use a GPU](https://www.tensorflow.org/guide/gpu)*

In [9]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


### Create the model

In [10]:
import nets

model = nets.MLP(n_classes=len(y_classes))

### Load the trained weights of the model parameters

In [11]:
WEIGHT_PATH = "runs_clf/train_20230506_183726/best"
model.load_weights(WEIGHT_PATH)

### The testing process

In [12]:
y_true = []
y_pred = []
for i, vdata in tqdm(enumerate(ds_test)):
    vinputs, vlabels = vdata
    y_true.extend(vlabels.numpy().tolist())
    # We don't need gradients for the model testing process
    # training=False is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    voutputs = model(vinputs, training=False)
    # the class with the highest energy is what we choose as prediction
    predicted = tf.math.argmax(voutputs, axis=1)
    y_pred.extend(predicted.numpy().tolist())

# Calculate our classification metrics
acc = accuracy_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred, average="weighted")
print(f'Accuracy: {acc}, Weighted F1: {f1}')


2023-05-06 20:34:42.768026: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int64 and shape [835]
	 [[{{node Placeholder/_1}}]]


0it [00:00, ?it/s]

Accuracy: 0.25269461077844313, Weighted F1: 0.23467965772123126
